In [570]:
import numpy as np
import matplotlib.pyplot as plt

In [571]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=5000)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
5000
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [572]:
def predict_train(input_str,
                  letters,
                  max_history=4,
                  max_index: int = 4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                # idx_strength = W[prev_inp + str(prev_indices[h_idx])][inp][inp_has_index]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
        else:
            new_index = max(IdxO, key=IdxO.get)
        WIN[inp][new_index] += 1

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)
            HistoryW[h_idx][key_I][inp][new_index] += 1

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW, HistoryHW, IDX

max_index_L1 = 32
max_history_L1 = 11

max_history_L1_ds = 11

HistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1, max_index=max_index_L1)

for h_idx in range(max_history_L1):
    print(f'{h_idx=}')
    for wk, wv in HistoryW[h_idx].items():
        print(wk, wv)


h_idx=0
c0 {'h': {0: 1}}
h0 {'a': {0: 1}, 'o': {4: 1}, 'e': {7: 1}, ' ': {12: 1}, 'i': {30: 1}}
a0 {'p': {0: 1}, 't': {7: 1}, 'i': {18: 1}, 'n': {2: 1}}
p0 {'t': {0: 1}, 'e': {6: 1}}
t0 {'e': {0: 1}, ' ': {2: 1}}
e0 {'r': {0: 2, 25: 1}, 'c': {12: 1}, ' ': {16: 3, 13: 1}, 't': {4: 1}, 'd': {9: 2}, 's': {22: 1}}
r0 {' ': {0: 1}, 'h': {20: 1}}
 0 {'i': {0: 2}, 'b': {7: 1}, 't': {19: 2}, 'a': {20: 1}, 'l': {1: 1}, 'y': {10: 1}, 'h': {7: 1}, 'g': {4: 1}}
i0 {'n': {0: 1}, ' ': {1: 1}}
n0 {' ': {1: 11, 14: 2}, 'o': {2: 1}, 'g': {0: 3, 15: 1}, 'c': {13: 1}, 'e': {20: 1}, "'": {2: 1}}
 1 {'w': {0: 3}, 'c': {5: 2, 6: 1}, 'o': {24: 2, 2: 1}, 't': {31: 5, 23: 1}, 'y': {10: 1}, 'i': {9: 4, 11: 1, 23: 1}, 's': {17: 2, 14: 1}, 'g': {10: 1}, 'n': {5: 1}, 'a': {21: 1, 12: 1}, 'r': {5: 1}, 'b': {21: 2}, 'h': {29: 2}, 'j': {4: 1}}
w0 {'h': {1: 3}, 'a': {15: 1}}
h1 {'i': {1: 1}, 'r': {8: 1}, 'o': {31: 2, 15: 2}, 'e': {13: 4}, 'a': {7: 1}}
i1 {'c': {1: 1, 6: 1}, 'd': {21: 5}, 's': {1: 3, 27: 1, 9: 1}, 'n':

In [573]:
def predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    AL = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}
        HO_cnt: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            # print(f'{h_idx=}')

            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                HO_cnt.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)
                    HO_cnt[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index
                    HO_cnt[key_O][key_index] += 1

        # print(f'{HO=}')
        # print(f'{HO_cnt=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        max_arr = []
        for v in HO.values():
            col_max = max(v, key=v.get)
            max_arr.append(col_max)
        
        max_arr = np.array(max_arr)
        # tf = np.mean(np.abs(max_arr - max_arr[np.newaxis].T))

        tf = length
        # tf = len(HO)

        TF.append(tf)

        # max by cnt
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO_cnt.items():
            max_idx = max(val, key=val.get)
            # HO_cnt[key] = {max_idx: val[max_idx]}

            if val[max_idx] > max_val: # 
                max_val = val[max_idx]
                max_symbol_index = max_idx
                max_symbol = key
        # print(f'{max_symbol=}')
        # max by val
        # max_symbol = 'a'
        # max_symbol_index = 0
        # max_val = 0
        # win_cnt = 0
        # for key, val in HO.items():
        #     max_idx = max(val, key=val.get)
        #     HO[key] = {max_idx: val[max_idx]}

        #     max_cnt = HO_cnt[key][max_idx]

        #     if val[max_idx] > max_val and max_cnt >= max_his:
        #         print(f'{key=} {val[max_idx]=} {max_cnt=} {max_his=}')
        #         max_val = val[max_idx]
        #         max_symbol_index = max_idx
        #         max_symbol = key
        #         win_cnt = max_cnt

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=} {max_val=}', max_symbol_index, max_val)
        # print('')

        # anomaly
        if max_symbol == next_inp:
            AL.append(0)
        else:
            ni_v = HO.get(next_inp, 0)
            if ni_v == 0:
                AL.append(100)
            else:
                max_ni = max(ni_v, key=ni_v.get)
                AL.append(abs(ni_v[max_ni] - max_val))

        new_indices = [max_symbol_index]

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    TF.append(0)
    AL.append(0)
    return err, TF, AL

err_L1, TF_L1, AL_L1 = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)
# err_L1, TF_L1_no, AL_L1_no = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)

print(err_L1, err_L1 / (len(input_str) / 100))

max_symbol='h' next_inp='h' max_val=1 0 1
max_symbol='a' next_inp='a' max_val=2 0 2
max_symbol='p' next_inp='p' max_val=3 0 3
max_symbol='t' next_inp='t' max_val=4 0 4
max_symbol='e' next_inp='e' max_val=5 0 5
max_symbol='r' next_inp='r' max_val=6 0 6
max_symbol=' ' next_inp=' ' max_val=7 0 7
max_symbol='i' next_inp='i' max_val=8 0 8
max_symbol='n' next_inp='n' max_val=9 0 9
max_symbol=' ' next_inp=' ' max_val=10 1 10
max_symbol='w' next_inp='w' max_val=11 0 11
max_symbol='h' next_inp='h' max_val=11 1 11
max_symbol='i' next_inp='i' max_val=11 1 11
max_symbol='c' next_inp='c' max_val=11 1 11
max_symbol='h' next_inp='h' max_val=11 2 11
max_symbol=' ' next_inp=' ' max_val=11 2 11
max_symbol='w' next_inp='w' max_val=11 1 11
max_symbol='e' next_inp='e' max_val=11 1 11
max_symbol=' ' next_inp=' ' max_val=11 3 11
max_symbol='a' next_inp='a' max_val=11 1 11
max_symbol='r' next_inp='r' max_val=11 1 11
max_symbol='e' next_inp='e' max_val=11 2 11
max_symbol=' ' next_inp=' ' max_val=11 4 11
max_sy

In [574]:
def get_word_stats(input_str: str | list, TF: list, threshold=0.0):
    buffer: list = [input_str[0]]
    WordStats: dict[str, int] = {}
    WordIndex: dict[str, int] = {}
    next_wrd_idx = 0

    NextLayerOut = []
    DownstreamIndex = {}

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 and inp_idx >= len(input_str) - 1:
            continue

        # if TF[inp_idx] - prev_tf > 0.0: # and abs(TF[inp_idx] - prev_tf) > 0.2:
        # and TF[inp_idx] > threshold:
        if TF[inp_idx] > prev_tf and TF[inp_idx] > TF[inp_idx + 1]:
            word = ''.join(buffer)

            WordStats.setdefault(word, 0)
            WordStats[word] += 1

            if not WordIndex.get(word):
                WordIndex.setdefault(word, next_wrd_idx)
                next_wrd_idx += 1

            wrd_idx = WordIndex.get(word)
            # NextLayerOut.append(str(wrd_idx))
            NextLayerOut.append(word)

            DownstreamIndex.setdefault(inp_idx, 1)

            buffer = [inp]
            prev_tf = TF[inp_idx]

            continue

        TF[inp_idx] = 0

        buffer.append(inp)
        prev_tf = TF[inp_idx]

    # sort
    WordStats = dict(sorted(WordStats.items(), key=lambda item: item[1], reverse=True))

    NextLayerOut.append(str(next_wrd_idx))
    return WordStats, WordIndex, NextLayerOut, DownstreamIndex


TF_L1 = np.array(TF_L1, dtype='float64')
AL_L1 = np.array(AL_L1, dtype='float64')
TF_L1 /= np.max(TF_L1)
AL_L1 /= np.max(AL_L1)
TF_L1 += AL_L1

TF_max_coef = 0.4

# TF_L1[np.where(TF_L1 < np.mean(TF_L1))] = 0
# TF_1 = TF_L1 - np.pad(TF_L1, (1, 0))[:-1]
# TF_2 = TF_L1 - np.pad(TF_L1, (0, 1))[1:]
# TF_L1 = TF_1 + TF_2

# TF_L1[np.where(TF_L1 < 0)] = 0
# TF_L1[np.where(TF_L1 < TF_max_coef)] = 0
TF_L1 = TF_L1.tolist()

WordStats_L1, WordIndex_L1, NextLayerOut_L1, DownstreamIndex_L1 = get_word_stats(input_str=input_str,
                                                                                 TF=TF_L1, 
                                                                                 threshold=0.0, )
print(len(WordStats_L1))
mean_WS = np.mean(np.array(list(WordStats_L1.values())))
print(f'{mean_WS=}')
WS_len = 0
for key, val in WordStats_L1.items():
    # if len(key.strip()) > 1 and val >= mean_WS:
    if val >= mean_WS:
        WS_len += 1
    print(key, val)
print(f'{WS_len=}')

print(f'{NextLayerOut_L1=}')
print(f'{len(NextLayerOut_L1)=}')

print(f'{WordIndex_L1=}')

print(f'{DownstreamIndex_L1=}')


815
mean_WS=2.1386503067484663
 a 44
 t 38
 b 35
 h 31
he 20
 s 19
nd 18
 w 18
 i 17
ng 14
 o 13
 f 13
 c 13
 he 12
er 12
 th 12
the 11
on 11
ou 11
 y 11
th 10
d  10
 p 10
e  10
in 9
is 9
 r 9
 the 9
 l 9
 n 8
at 8
st 8
ll 8
nd  7
ie 7
ow 7
oph 7
 d 7
y  7
 to 7
 g 7
an 7
as 7
 be 6
es 6
ng  6
of 6
 sa 6
all 6
im 6
aid 6
ooh 6
t  6
s  6
 li 6
re 5
rs 5
ai 5
or 5
't 5
to 5
oh 5
en 5
ha 5
ut 5
hat 5
f  5
ob 5
r  5
uzz 5
 m 5
ed 5
ra 5
hr 5
and 4
ar 4
 co 4
 ch 4
nt 4
so 4
n  4
 you 4
id 4
o  4
 e 4
oi 4
 of 4
his 4
ime 4
ong 4
ea 4
ist 4
le 4
uz 4
el 4
li 4
t i 4
is  4
ther 4
ur 4
nch 4
we 3
 in 3
 po 3
ome 3
e s 3
mi 3
 bu 3
mp 3
 as 3
nly 3
no 3
ly 3
ny 3
ro 3
 wi 3
id  3
ay 3
bo 3
 k 3
u  3
do 3
be 3
es  3
po 3
ame 3
ns 3
 an 3
tly 3
sai 3
ke 3
was 3
ing  3
he  3
f t 3
t t 3
ir 3
ing 3
ni 3
om 3
ki 3
ee 3
 ho 3
un 3
et 3
ot 3
ood 3
al 3
tor 2
re  2
ind 2
t is 2
 but 2
at  2
bu 2
 of  2
eels 2
 at 2
ead 2
ed t 2
o y 2
nn 2
e i 2
e w 2
obin 2
e's 2
inn 2
her 2
now  2
wh 2
 ' 2
now 2
 q 

In [575]:
def downstream_train(input: list, next_layer_out: list, downstream_index: dict[int, int]):
    DSStats = {}

    ds_idx = 1
    ds_inp = next_layer_out[0]

    for inp_idx, inp in enumerate(input[:-1]):
        has_ds_input = downstream_index.get(inp_idx)
        if has_ds_input:
            ds_inp = next_layer_out[ds_idx]
            ds_idx += 1

        ds_key = ds_inp + ':' + inp
        DSStats.setdefault(ds_key, 0)
        DSStats[ds_key] += 1

    return DSStats


DSStats = downstream_train(input=input_str,
                           next_layer_out=NextLayerOut_L1,
                           downstream_index=DownstreamIndex_L1,)

for wk, wv in DSStats.items():
    print(wk, wv)

cch:c 1
cch:h 1
apter in:a 1
apter in:p 1
apter in:t 1
apter in:e 1
apter in:r 1
apter in:  1
apter in:i 1
apter in:n 1
 whic:  1
 whic:w 1
 whic:h 1
 whic:i 1
 whic:c 1
h :h 1
h :  1
we:w 3
we:e 3
 a:  44
 a:a 44
re:r 5
re:e 5
 in:  3
 in:i 3
 in:n 3
tro:t 1
tro:r 1
tro:o 1
duced:d 2
duced:u 1
duced:c 1
duced:e 1
 t:  38
 t:t 38
o wi:o 1
o wi:  1
o wi:w 1
o wi:i 1
nnie :n 2
nnie :i 1
nnie :e 1
nnie :  1
the:t 11
the:h 11
the:e 11
 po:  3
 po:p 3
 po:o 3
oh :o 1
oh :h 1
oh :  1
and:a 4
and:n 4
and:d 4
 s:  19
 s:s 19
ome:o 3
ome:m 3
ome:e 3
 be:  6
 be:b 6
 be:e 6
es:e 6
es:s 6
nd :n 7
nd :d 7
nd :  7
th:t 10
th:h 10
e s:e 3
e s:  3
e s:s 3
tor:t 2
tor:o 2
tor:r 2
ie:i 7
ie:e 7
s b:s 1
s b:  1
s b:b 1
eg:e 1
eg:g 1
in:i 9
in:n 9
 he:  12
 he:h 12
 he:e 12
re :r 2
re :e 2
re :  2
is e:i 1
is e:s 1
is e:  1
is e:e 1
dw:d 1
dw:w 1
ar:a 4
ar:r 4
d be:d 1
d be:  1
d be:b 1
d be:e 1
 co:  4
 co:c 4
 co:o 4
mi:m 3
mi:i 3
ng :n 6
ng :g 6
ng :  6
downstai:d 1
downstai:o 1
downstai:w 1
downstai:

In [576]:
def downstream_predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            upstream_inp: dict[int, list],
            DSStats: dict[str, int],
            ds_input: list,
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    ds_idx = 1
    ds_inp = ds_input[0]

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            # print(f'{key_I=}')
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)
                # print(f'{key_I=}')

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        # tf = length
        tf = len(HO)

        TF.append(tf)

        # ds input
        has_ds_input = upstream_inp.get(input_idx + 1)
        if has_ds_input:
            ds_inp = ds_input[ds_idx]
            ds_idx += 1

        # max symbol
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}
            max_idx_val = val[max_idx]

            # ds predict
            ds_key = ds_inp + ':' + key
            ds_stats = DSStats.get(ds_key)
            # if ds_stats:
            #     max_idx_val += ds_stats

            if max_idx_val > max_val and ds_stats:
                max_val = max_idx_val
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=} {ds_inp=} {HO=}', max_symbol_index, max_val)

        # print(f'{O.get(max_symbol).keys()=}')
        new_indices = [max_symbol_index]

        if O.get(max_symbol):
            new_indices = list(O.get(max_symbol).keys())            

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    return err, TF

HistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1_ds, max_index=max_index_L1)

# for h_idx in range(max_history_L1_ds):
#     print(f'{h_idx=}')
#     for wk, wv in HistoryW[h_idx].items():
#         print(wk, wv)

err_L1_ds, TF_L1_ds = downstream_predict(input_str=input_str, 
                                        HistoryW=HistoryW, 
                                        upstream_inp=DownstreamIndex_L1,
                                        DSStats=DSStats,
                                        ds_input=NextLayerOut_L1,
                                        max_history=max_history_L1_ds,
                                        max_index=max_index_L1, )

print(f'{err_L1=}', err_L1 / (len(input_str) / 100))
print(f'{err_L1_ds=}', err_L1_ds / (len(input_str) / 100))

max_symbol='h' next_inp='h' ds_inp='cch' HO={'h': {29: 4}, 'e': {4: 3}, 'a': {27: 2}, 'o': {15: 4}, 'k': {4: 3}, 'r': {30: 1}, 'l': {4: 3}, 'i': {22: 1}} 29 4
max_symbol='e' next_inp='a' ds_inp='apter in' HO={'a': {5: 9}, 'o': {2: 3}, 'e': {16: 30}, ' ': {13: 3}, 'i': {9: 5}, 't': {8: 3}, 'r': {26: 9}, 'y': {25: 1}, 'd': {1: 1}, 'n': {24: 1}, 'm': {5: 3}, 'u': {14: 2}, 'l': {9: 3}, 'f': {0: 1}} 16 30
max_symbol='t' next_inp='p' ds_inp='apter in' HO={'r': {25: 10}, 't': {24: 13}, 'd': {13: 7}, 's': {31: 8}, 'n': {15: 7}, ' ': {10: 7}, 'c': {6: 2}, 'l': {10: 2}, 'p': {0: 3}, 'y': {19: 2}, 'w': {1: 2}, 'i': {2: 4}, "'": {2: 1}, 'u': {13: 1}, 'a': {1: 2}, 'v': {0: 1}, 'h': {29: 2}, 'b': {21: 2}, 'e': {13: 1}, 'o': {15: 2}, 'm': {0: 4}, 'g': {9: 1}} 24 13
max_symbol=' ' next_inp='t' ds_inp='apter in' HO={'t': {0: 3}, 'e': {11: 6}, ' ': {10: 21}, 'd': {13: 3}, 'l': {16: 6}, 'n': {15: 4}, 'g': {2: 4}, 'p': {24: 6}, 's': {14: 9}, 'h': {29: 5}, 'c': {13: 1}, 'w': {8: 3}, 'o': {5: 3}, 'r': {25: 